In [1]:
import pandas as pd

In [2]:
dirty_df = pd.read_csv("./Group113_dirty_data.csv")
outlier_df = pd.read_csv("./Group113_outlier_data.csv")
missing_df = pd.read_csv("./Group113_missing_data.csv")
node = pd.read_csv("./nodes.csv")
edges = pd.read_csv("./edges.csv")
branches = pd.read_csv("./branches.csv")

In [3]:
dirty_df.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,490.960700,-30.754358,143.504515,0.126000,8.822786,13.722854
std,258.944904,25.336872,16.299119,0.332182,1.648781,2.564382
min,43.000000,-37.840654,-37.823134,0.000000,3.562000,3.955939
25%,296.812500,-37.818644,144.950373,0.000000,7.882250,12.502573
50%,460.100000,-37.812013,144.963606,0.000000,8.872500,13.781359
75%,645.000000,-37.804556,144.982663,0.000000,9.954000,15.350428
max,1397.000000,145.015210,145.018834,1.000000,13.848000,20.370184


In [5]:
dirty_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
order_id                   500 non-null object
date                       500 non-null object
time                       500 non-null object
order_type                 500 non-null object
branch_code                500 non-null object
order_items                500 non-null object
order_price                500 non-null float64
customer_lat               500 non-null float64
customer_lon               500 non-null float64
customerHasloyalty?        500 non-null int64
distance_to_customer_KM    500 non-null float64
delivery_fee               500 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


In [7]:
dirty_df.describe(include=['O'])

,order_id,date,time,order_type,branch_code,order_items
count,500,500,500,500,500,500
unique,500,298,72,3,6,498
top,ORDX10966,2018-01-04,18:08:27,Dinner,TP,"[('Fish&Chips', 7), ('Pasta', 3)]"
freq,1,6,13,170,174,2


In [13]:
dirty_df.head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDY03163,2018-09-15,16:27:02,Dinner,tp,"[('Fish&Chips', 4), ('Salmon', 7), ('Pasta', 3...",671.5,-37.815611,144.973490,0,8.318,14.140324
1,ORDB08039,2018-05-19,12:33:48,Lunch,TP,"[('Steak', 4), ('Salad', 7)]",300.4,-37.806544,145.010707,0,12.438,17.011135
2,ORDA01841,2018-08-21,15:36:20,Lunch,BK,"[('Fish&Chips', 3), ('Fries', 10), ('Chicken',...",511.0,-37.823347,144.988505,0,6.383,12.206218
3,ORDB05089,2018-03-14,10:21:58,Breakfast,TP,"[('Coffee', 3), ('Cereal', 10), ('Pancake', 10)]",475.0,-37.799659,144.970194,0,9.221,12.911713
4,ORDA10328,2018-07-16,14:25:21,Dinner,BK,"[('Fries', 9), ('Salad', 8), ('Burger', 6)]",431.6,-37.805799,144.994800,0,5.102,11.231360


In [48]:
# checking date format

mask = pd.to_datetime(dirty_df.date, format='%Y-%m-%d', errors='coerce').isna()
dirty_date = dirty_df.loc[mask, :]
dirty_date

In [50]:
# fixing date format
def fix_date(date_str):
    date_token = date_str.split('-')
    year = (date_token[0])
    month = (int)(date_token[1])
    date = (int)(date_token[2])
    
    if len(year) != 4:
        year,date = date,(int)(year)
    
    if month > 12:
        date,month = month,date

    if month in (4,6,9,11):
        if date >30:
            date = 30
            
    if month == 2:
        if date>28:
            date =28
    
    final_date = pd.to_datetime(str(year)+"-"+str(month)+"-"+str(date),format='%Y-%m-%d')
#     print("date string:" + date_str + "######" + " final date:" + str(final_date))
    return final_date
        
dirty_date['date']=dirty_date['date'].apply(fix_date)
dirty_date

date string:2018-24-09###### final date:2018-09-24 00:00:00
date string:10-01-2018###### final date:2018-01-10 00:00:00
date string:2018-23-05###### final date:2018-05-23 00:00:00
date string:2018-16-10###### final date:2018-10-16 00:00:00
date string:2018-19-01###### final date:2018-01-19 00:00:00
date string:04-11-2018###### final date:2018-11-04 00:00:00
date string:2018-15-08###### final date:2018-08-15 00:00:00
date string:11-11-2018###### final date:2018-11-11 00:00:00
date string:2018-17-07###### final date:2018-07-17 00:00:00
date string:2018-20-12###### final date:2018-12-20 00:00:00
date string:03-06-2018###### final date:2018-06-03 00:00:00
date string:2018-31-08###### final date:2018-08-31 00:00:00
date string:2018-26-04###### final date:2018-04-26 00:00:00
date string:12-11-2018###### final date:2018-11-12 00:00:00
date string:02-02-2018###### final date:2018-02-02 00:00:00
date string:2018-14-09###### final date:2018-09-14 00:00:00
date string:05-03-2018###### final date:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
30,ORDC04608,2018-09-24,16:47:19,Dinner,NS,"[('Shrimp', 10), ('Pasta', 2), ('Fish&Chips', 3)]",700.00,-37.816886,144.988144,0,9.050,15.276973
40,ORDX10324,2018-01-10,08:10:08,Breakfast,BK,"[('Eggs', 3), ('Pancake', 5), ('Cereal', 9)]",376.25,-37.806651,144.957028,1,8.296,6.822306
50,ORDJ06852,2018-05-23,12:43:56,Lunch,TP,"[('Fries', 5), ('Steak', 10), ('Burger', 4)]",634.00,-37.814628,144.983456,0,9.310,13.094294
55,ORDB05508,2018-10-16,17:27:53,Dinner,TP,"[('Pasta', 8), ('Shrimp', 5), ('Fish&Chips', 8...",1180.00,-37.822314,144.996798,0,10.154,14.085795
61,ORDB09528,2018-01-19,08:30:25,Breakfast,TP,"[('Eggs', 4), ('Coffee', 9), ('Pancake', 3)]",228.25,-37.805573,144.953490,0,9.413,11.779022
80,ORDX10265,2018-11-04,18:08:27,Dinner,BK,"[('Pasta', 6), ('Shrimp', 8), ('Fish&Chips', 7...",965.00,-37.811351,144.952810,0,8.456,17.851977
153,ORDI09232,2018-08-15,15:26:11,Lunch,NS,"[('Fries', 4), ('Steak', 2), ('Chicken', 10), ...",692.20,-37.808185,144.975624,0,7.292,12.705843
164,ORDX01295,2018-11-11,18:28:43,Dinner,BK,"[('Salmon', 8), ('Shrimp', 8), ('Pasta', 5), (...",967.50,-37.817193,144.958842,0,8.285,17.801597
166,ORDC00567,2018-07-17,14:25:21,Lunch,NS,"[('Salad', 10), ('Chicken', 6), ('Burger', 9)]",643.00,-37.808679,144.948694,0,8.809,15.180942
167,ORDI08693,2018-12-20,19:39:43,Dinner,NS,"[('Fish&Chips', 10), ('Shrimp', 5)]",620.00,-37.815358,144.936667,0,10.042,15.928873


In [69]:
dirty_new = dirty_df[~dirty_df['order_id'].isin(dirty_date['order_id'])]

In [70]:
len(dirty_new)

463

In [71]:
# check time format

mask1 = pd.to_datetime(dirty_new.time, format='%H:%M:%S', errors='coerce').isna()
dirty_time = dirty_new.loc[mask1, :]
dirty_time

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee


In [72]:
# checking branch code
dirty_new.branch_code.value_counts()

TP    162
BK    140
NS    135
tp     12
bk      7
ns      7
Name: branch_code, dtype: int64

In [73]:
dirty_bc = dirty_new[(dirty_new.branch_code == 'tp') | (dirty_new.branch_code == 'bk') |(dirty_new.branch_code == 'ns')] 
dirty_bc

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDY03163,2018-09-15,16:27:02,Dinner,tp,"[('Fish&Chips', 4), ('Salmon', 7), ('Pasta', 3...",671.50,-37.815611,144.973490,0,8.318,14.140324
13,ORDK00414,2018-06-27,13:44:47,Lunch,tp,"[('Salad', 8), ('Fries', 9), ('Chicken', 3)]",341.60,-37.818929,144.973552,0,7.245,13.235827
28,ORDJ00251,2018-12-11,19:19:26,Dinner,ns,"[('Fish&Chips', 1), ('Shrimp', 1), ('Pasta', 8)]",309.00,-37.822558,144.950131,0,8.647,12.832132
98,ORDB04798,2018-06-03,13:04:13,Lunch,bk,"[('Burger', 1), ('Chicken', 10), ('Fries', 7),...",607.00,-37.816684,144.970055,0,7.984,12.902967
107,ORDA10203,2018-04-19,11:32:57,Breakfast,tp,"[('Eggs', 4), ('Cereal', 1), ('Pancake', 6), (...",314.50,-37.813404,144.974073,0,6.806,11.718539
132,ORDJ06622,2018-05-29,12:54:05,Lunch,ns,"[('Chicken', 7), ('Fries', 5), ('Steak', 6), (...",970.60,-37.816534,145.008947,0,11.625,14.418109
154,ORDK05097,2018-07-23,14:45:38,Lunch,ns,"[('Salad', 4), ('Steak', 3), ('Fries', 9)]",311.80,-37.823694,145.013756,0,4.828,10.575452
188,ORDB08700,2018-08-22,15:46:28,Lunch,tp,"[('Fries', 5), ('Steak', 6)]",330.00,-37.820425,144.976665,0,8.511,11.914621
235,ORDB03709,2018-07-07,14:15:12,Lunch,tp,"[('Chicken', 7), ('Steak', 8)]",584.00,-37.818630,145.016065,0,12.751,17.293692
239,ORDI09901,2018-07-01,13:54:55,Lunch,ns,"[('Steak', 1), ('Chicken', 1)]",77.00,-37.813624,144.941056,0,9.513,16.408801


In [79]:
# fixing branch code
dirty_bc['branch_code'] = dirty_bc.apply(lambda row: row['branch_code'].upper(),axis=1)
dirty_bc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDY03163,2018-09-15,16:27:02,Dinner,TP,"[('Fish&Chips', 4), ('Salmon', 7), ('Pasta', 3...",671.50,-37.815611,144.973490,0,8.318,14.140324
13,ORDK00414,2018-06-27,13:44:47,Lunch,TP,"[('Salad', 8), ('Fries', 9), ('Chicken', 3)]",341.60,-37.818929,144.973552,0,7.245,13.235827
28,ORDJ00251,2018-12-11,19:19:26,Dinner,NS,"[('Fish&Chips', 1), ('Shrimp', 1), ('Pasta', 8)]",309.00,-37.822558,144.950131,0,8.647,12.832132
98,ORDB04798,2018-06-03,13:04:13,Lunch,BK,"[('Burger', 1), ('Chicken', 10), ('Fries', 7),...",607.00,-37.816684,144.970055,0,7.984,12.902967
107,ORDA10203,2018-04-19,11:32:57,Breakfast,TP,"[('Eggs', 4), ('Cereal', 1), ('Pancake', 6), (...",314.50,-37.813404,144.974073,0,6.806,11.718539
132,ORDJ06622,2018-05-29,12:54:05,Lunch,NS,"[('Chicken', 7), ('Fries', 5), ('Steak', 6), (...",970.60,-37.816534,145.008947,0,11.625,14.418109
154,ORDK05097,2018-07-23,14:45:38,Lunch,NS,"[('Salad', 4), ('Steak', 3), ('Fries', 9)]",311.80,-37.823694,145.013756,0,4.828,10.575452
188,ORDB08700,2018-08-22,15:46:28,Lunch,TP,"[('Fries', 5), ('Steak', 6)]",330.00,-37.820425,144.976665,0,8.511,11.914621
235,ORDB03709,2018-07-07,14:15:12,Lunch,TP,"[('Chicken', 7), ('Steak', 8)]",584.00,-37.818630,145.016065,0,12.751,17.293692
239,ORDI09901,2018-07-01,13:54:55,Lunch,NS,"[('Steak', 1), ('Chicken', 1)]",77.00,-37.813624,144.941056,0,9.513,16.408801


In [82]:
# fixing branch code
def check_branch_code(order_id, branch_code):
    if order_id.startswith('ORDA') or order_id.startswith('ORDK') or order_id.startswith('ORDX'):
        return 'BK'
    elif order_id.startswith('ORDB') or order_id.startswith('ORDJ') or order_id.startswith('ORDY'):
        return 'TP'
    elif order_id.startswith('ORDC') or order_id.startswith('ORDI') or order_id.startswith('ORDZ'):
        return 'NS'
    return branch_code

In [83]:
dirty_bc['branch_code'] = dirty_bc.apply(lambda row:check_branch_code(row['order_id'], row['branch_code']),axis=1)
dirty_bc

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDY03163,2018-09-15,16:27:02,Dinner,TP,"[('Fish&Chips', 4), ('Salmon', 7), ('Pasta', 3...",671.50,-37.815611,144.973490,0,8.318,14.140324
13,ORDK00414,2018-06-27,13:44:47,Lunch,BK,"[('Salad', 8), ('Fries', 9), ('Chicken', 3)]",341.60,-37.818929,144.973552,0,7.245,13.235827
28,ORDJ00251,2018-12-11,19:19:26,Dinner,TP,"[('Fish&Chips', 1), ('Shrimp', 1), ('Pasta', 8)]",309.00,-37.822558,144.950131,0,8.647,12.832132
98,ORDB04798,2018-06-03,13:04:13,Lunch,TP,"[('Burger', 1), ('Chicken', 10), ('Fries', 7),...",607.00,-37.816684,144.970055,0,7.984,12.902967
107,ORDA10203,2018-04-19,11:32:57,Breakfast,BK,"[('Eggs', 4), ('Cereal', 1), ('Pancake', 6), (...",314.50,-37.813404,144.974073,0,6.806,11.718539
132,ORDJ06622,2018-05-29,12:54:05,Lunch,TP,"[('Chicken', 7), ('Fries', 5), ('Steak', 6), (...",970.60,-37.816534,145.008947,0,11.625,14.418109
154,ORDK05097,2018-07-23,14:45:38,Lunch,BK,"[('Salad', 4), ('Steak', 3), ('Fries', 9)]",311.80,-37.823694,145.013756,0,4.828,10.575452
188,ORDB08700,2018-08-22,15:46:28,Lunch,TP,"[('Fries', 5), ('Steak', 6)]",330.00,-37.820425,144.976665,0,8.511,11.914621
235,ORDB03709,2018-07-07,14:15:12,Lunch,TP,"[('Chicken', 7), ('Steak', 8)]",584.00,-37.818630,145.016065,0,12.751,17.293692
239,ORDI09901,2018-07-01,13:54:55,Lunch,NS,"[('Steak', 1), ('Chicken', 1)]",77.00,-37.813624,144.941056,0,9.513,16.408801


In [75]:
dirty_new_1 = dirty_new[~dirty_new['order_id'].isin(dirty_bc['order_id'])]
len(dirty_new_1)

437

I am not checking it on the extracted data as we will anyway run it on the entire data

In [85]:
# dirty_new_1[((dirty_new_1.order_id.startswith('ORDA') or dirty_new_1.order_id.startswith('ORDK') or dirty_new_1.order_id.startswith('ORDX')) \
#             and (dirty_new_1.branch_code != 'BK'))|\
#             ((dirty_new_1.order_id.startswith('ORDB') or dirty_new_1.order_id.startswith('ORDJ') or dirty_new_1.order_id.startswith('ORDY')) \
#             and (dirty_new_1.branch_code != 'TP'))|\
#             ((dirty_new_1.order_id.startswith('ORDC') or dirty_new_1.order_id.startswith('ORDI') or dirty_new_1.order_id.startswith('ORDZ')) \
#             and (dirty_new_1.branch_code != 'NS'))]

In [96]:

# checking interchange latitude longitude
interchanged_df = dirty_new_1[(dirty_new_1['customer_lat'] > 0) & (dirty_new_1['customer_lon'] < 0)]

idx = (interchanged_df['customer_lat'] > 0) & (interchanged_df['customer_lon'] < 0)

interchanged_df

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
376,ORDX10477,2018-08-06,15:05:54,Lunch,BK,"[('Steak', 3), ('Burger', 7), ('Salad', 4), ('...",540.8,144.970391,-37.823134,0,7.938,13.555022
385,ORDJ02390,2018-11-05,18:08:27,Dinner,TP,"[('Pasta', 6), ('Salmon', 10), ('Fish&Chips', ...",1160.0,144.963469,-37.815258,0,7.906,12.166725
459,ORDA05768,2018-05-19,12:33:48,Lunch,BK,"[('Steak', 5), ('Salad', 10), ('Fries', 1), ('...",817.0,144.962735,-37.803438,0,7.742,16.251153
460,ORDZ08685,2018-12-06,19:19:26,Dinner,NS,"[('Salmon', 8), ('Pasta', 5)]",465.5,145.015210,-37.810352,0,10.826,16.713025


In [97]:
# fixing latitude longitude
interchanged_df.loc[idx,['customer_lat','customer_lon']] = interchanged_df.loc[idx,['customer_lon','customer_lat']].values
interchanged_df

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
376,ORDX10477,2018-08-06,15:05:54,Lunch,BK,"[('Steak', 3), ('Burger', 7), ('Salad', 4), ('...",540.8,-37.823134,144.970391,0,7.938,13.555022
385,ORDJ02390,2018-11-05,18:08:27,Dinner,TP,"[('Pasta', 6), ('Salmon', 10), ('Fish&Chips', ...",1160.0,-37.815258,144.963469,0,7.906,12.166725
459,ORDA05768,2018-05-19,12:33:48,Lunch,BK,"[('Steak', 5), ('Salad', 10), ('Fries', 1), ('...",817.0,-37.803438,144.962735,0,7.742,16.251153
460,ORDZ08685,2018-12-06,19:19:26,Dinner,NS,"[('Salmon', 8), ('Pasta', 5)]",465.5,-37.810352,145.015210,0,10.826,16.713025


In [98]:
dirty_new_2 = dirty_new_1[~dirty_new_1['order_id'].isin(interchanged_df['order_id'])]
len(dirty_new_2)

433

In [101]:

# checking positive latitude 
positive_lat_to_negative = dirty_new_2[dirty_new_2['customer_lat'] > 0]
positive_lat_to_negative

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
11,ORDJ04320,2018-06-29,13:54:55,Lunch,TP,"[('Fries', 3), ('Steak', 8)]",396.00,37.806172,144.944749,0,10.048,13.321015
15,ORDC08350,2018-04-04,11:02:32,Breakfast,NS,"[('Pancake', 4), ('Eggs', 10), ('Cereal', 4), ...",446.00,37.817408,144.945824,0,9.435,14.395185
27,ORDK05494,2018-11-05,18:08:27,Dinner,BK,"[('Shrimp', 5), ('Fish&Chips', 1)]",305.00,37.819857,144.951875,0,8.922,16.059124
31,ORDZ03638,2018-11-07,18:18:35,Dinner,NS,"[('Salmon', 7), ('Pasta', 9), ('Shrimp', 10), ...",1354.50,37.811267,144.961717,0,7.744,13.626144
34,ORDZ02029,2018-05-04,12:03:22,Lunch,NS,"[('Burger', 6), ('Salad', 7), ('Fries', 9)]",414.40,37.800157,144.935366,0,6.636,12.369558
45,ORDY08080,2018-11-28,18:59:09,Dinner,TP,"[('Shrimp', 6), ('Fish&Chips', 7)]",569.00,37.805046,144.967249,0,9.401,13.727322
48,ORDC06702,2018-05-12,12:13:31,Lunch,NS,"[('Salad', 2), ('Fries', 8), ('Burger', 2)]",192.40,37.802282,144.962224,0,7.018,14.787773
57,ORDZ08200,2018-01-03,08:00:00,Breakfast,NS,"[('Coffee', 7), ('Eggs', 1), ('Cereal', 4)]",158.50,37.800937,145.015453,1,10.778,8.221095
63,ORDC09613,2018-08-12,15:26:11,Lunch,NS,"[('Chicken', 8), ('Fries', 8), ('Steak', 5), (...",738.80,37.818178,144.964958,0,8.504,15.937463
67,ORDZ00943,2018-04-06,11:02:32,Breakfast,NS,"[('Pancake', 7), ('Cereal', 6), ('Eggs', 6)]",427.75,37.812266,144.983811,0,8.327,13.559941


In [102]:
# fixing positive latitude to negative
positive_lat_to_negative['customer_lat'] = -positive_lat_to_negative['customer_lat']
positive_lat_to_negative

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
11,ORDJ04320,2018-06-29,13:54:55,Lunch,TP,"[('Fries', 3), ('Steak', 8)]",396.00,-37.806172,144.944749,0,10.048,13.321015
15,ORDC08350,2018-04-04,11:02:32,Breakfast,NS,"[('Pancake', 4), ('Eggs', 10), ('Cereal', 4), ...",446.00,-37.817408,144.945824,0,9.435,14.395185
27,ORDK05494,2018-11-05,18:08:27,Dinner,BK,"[('Shrimp', 5), ('Fish&Chips', 1)]",305.00,-37.819857,144.951875,0,8.922,16.059124
31,ORDZ03638,2018-11-07,18:18:35,Dinner,NS,"[('Salmon', 7), ('Pasta', 9), ('Shrimp', 10), ...",1354.50,-37.811267,144.961717,0,7.744,13.626144
34,ORDZ02029,2018-05-04,12:03:22,Lunch,NS,"[('Burger', 6), ('Salad', 7), ('Fries', 9)]",414.40,-37.800157,144.935366,0,6.636,12.369558
45,ORDY08080,2018-11-28,18:59:09,Dinner,TP,"[('Shrimp', 6), ('Fish&Chips', 7)]",569.00,-37.805046,144.967249,0,9.401,13.727322
48,ORDC06702,2018-05-12,12:13:31,Lunch,NS,"[('Salad', 2), ('Fries', 8), ('Burger', 2)]",192.40,-37.802282,144.962224,0,7.018,14.787773
57,ORDZ08200,2018-01-03,08:00:00,Breakfast,NS,"[('Coffee', 7), ('Eggs', 1), ('Cereal', 4)]",158.50,-37.800937,145.015453,1,10.778,8.221095
63,ORDC09613,2018-08-12,15:26:11,Lunch,NS,"[('Chicken', 8), ('Fries', 8), ('Steak', 5), (...",738.80,-37.818178,144.964958,0,8.504,15.937463
67,ORDZ00943,2018-04-06,11:02:32,Breakfast,NS,"[('Pancake', 7), ('Cereal', 6), ('Eggs', 6)]",427.75,-37.812266,144.983811,0,8.327,13.559941


In [104]:
dirty_new_3 = dirty_new_2[~dirty_new_2['order_id'].isin(positive_lat_to_negative['order_id'])]
len(dirty_new_3)

396

In [106]:
# No customer longitude with negative sign

dirty_new_3[dirty_new_3['customer_lon'] < 0]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee


In [108]:
# checking order type with respect to time
time_order_type = dirty_new_3[((dirty_new_3.time >= '08:00:00') & (dirty_new_3.time <= '12:00:00') & (dirty_new_3.order_type != 'Breakfast'))|\
            ((dirty_new_3.time >= '12:00:01') & (dirty_new_3.time <= '16:00:00') & (dirty_new_3.order_type != 'Lunch'))|\
            ((dirty_new_3.time >= '16:00:01') & (dirty_new_3.time <= '20:00:00') & (dirty_new_3.order_type != 'Dinner'))]


time_order_type

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
4,ORDA10328,2018-07-16,14:25:21,Dinner,BK,"[('Fries', 9), ('Salad', 8), ('Burger', 6)]",431.60,-37.805799,144.994800,0,5.102,11.231360
49,ORDX06996,2018-06-15,13:24:30,Dinner,BK,"[('Burger', 2), ('Fries', 8)]",158.00,-37.813268,144.965470,0,7.520,12.629953
75,ORDZ04960,2018-06-09,13:14:21,Dinner,NS,"[('Burger', 6), ('Chicken', 8)]",442.00,-37.818399,144.978798,0,9.109,16.502571
81,ORDC04467,2018-02-02,09:00:50,Dinner,NS,"[('Pancake', 10), ('Cereal', 9), ('Coffee', 8)...",513.50,-37.819764,145.012797,0,11.284,16.189410
105,ORDZ08634,2018-02-15,09:21:07,Dinner,NS,"[('Eggs', 7), ('Cereal', 9), ('Pancake', 8)]",537.00,-37.811941,144.969361,0,7.441,12.513105
116,ORDZ08328,2018-04-21,11:32:57,Dinner,NS,"[('Coffee', 8), ('Eggs', 9), ('Cereal', 4), ('...",390.50,-37.818818,144.953376,0,9.227,16.908111
118,ORDY08030,2018-09-26,16:47:19,Lunch,TP,"[('Shrimp', 9), ('Pasta', 8), ('Fish&Chips', 2...",899.00,-37.799264,144.960783,0,10.011,13.816681
126,ORDC05978,2018-03-10,10:11:49,Lunch,NS,"[('Pancake', 5), ('Coffee', 1), ('Cereal', 2)]",170.75,-37.822858,145.014101,0,11.710,19.117573
133,ORDB01502,2018-12-06,19:09:17,Breakfast,TP,"[('Pasta', 10), ('Fish&Chips', 2), ('Salmon', ...",643.00,-37.818440,144.956745,0,8.155,13.288764
147,ORDJ03556,2018-12-01,19:09:17,Lunch,TP,"[('Pasta', 6), ('Shrimp', 9), ('Fish&Chips', 1)]",686.00,-37.808319,144.936399,0,10.529,15.613354


In [110]:
# fixing order type with respect to time
def fix_order_type_with_respect_to_time(time, order_type):
    if (time >= '08:00:00') & (time <= '12:00:00') & (order_type != 'Breakfast'):
        order_type = 'Breakfast'
    elif (time >= '12:00:01') & (time <= '16:00:00') & (order_type != 'Lunch'):
        order_type = 'Lunch'
    elif (time >= '16:00:01') & (time <= '20:00:00') & (order_type != 'Dinner'):
        order_type = 'Dinner'
    
    return order_type

time_order_type['order_type'] = time_order_type.apply(lambda row:fix_order_type_with_respect_to_time(row['time'], row['order_type']),axis=1)
time_order_type

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
4,ORDA10328,2018-07-16,14:25:21,Lunch,BK,"[('Fries', 9), ('Salad', 8), ('Burger', 6)]",431.60,-37.805799,144.994800,0,5.102,11.231360
49,ORDX06996,2018-06-15,13:24:30,Lunch,BK,"[('Burger', 2), ('Fries', 8)]",158.00,-37.813268,144.965470,0,7.520,12.629953
75,ORDZ04960,2018-06-09,13:14:21,Lunch,NS,"[('Burger', 6), ('Chicken', 8)]",442.00,-37.818399,144.978798,0,9.109,16.502571
81,ORDC04467,2018-02-02,09:00:50,Breakfast,NS,"[('Pancake', 10), ('Cereal', 9), ('Coffee', 8)...",513.50,-37.819764,145.012797,0,11.284,16.189410
105,ORDZ08634,2018-02-15,09:21:07,Breakfast,NS,"[('Eggs', 7), ('Cereal', 9), ('Pancake', 8)]",537.00,-37.811941,144.969361,0,7.441,12.513105
116,ORDZ08328,2018-04-21,11:32:57,Breakfast,NS,"[('Coffee', 8), ('Eggs', 9), ('Cereal', 4), ('...",390.50,-37.818818,144.953376,0,9.227,16.908111
118,ORDY08030,2018-09-26,16:47:19,Dinner,TP,"[('Shrimp', 9), ('Pasta', 8), ('Fish&Chips', 2...",899.00,-37.799264,144.960783,0,10.011,13.816681
126,ORDC05978,2018-03-10,10:11:49,Breakfast,NS,"[('Pancake', 5), ('Coffee', 1), ('Cereal', 2)]",170.75,-37.822858,145.014101,0,11.710,19.117573
133,ORDB01502,2018-12-06,19:09:17,Dinner,TP,"[('Pasta', 10), ('Fish&Chips', 2), ('Salmon', ...",643.00,-37.818440,144.956745,0,8.155,13.288764
147,ORDJ03556,2018-12-01,19:09:17,Dinner,TP,"[('Pasta', 6), ('Shrimp', 9), ('Fish&Chips', 1)]",686.00,-37.808319,144.936399,0,10.529,15.613354


In [111]:
dirty_new_4 = dirty_new_3[~dirty_new_3['order_id'].isin(time_order_type['order_id'])]
len(dirty_new_4)

359

In [8]:
missing_df.shape

(500, 12)

In [9]:
missing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
order_id                   500 non-null object
date                       500 non-null object
time                       500 non-null object
order_type                 500 non-null object
branch_code                400 non-null object
order_items                500 non-null object
order_price                500 non-null float64
customer_lat               500 non-null float64
customer_lon               500 non-null float64
customerHasloyalty?        500 non-null int64
distance_to_customer_KM    450 non-null float64
delivery_fee               450 non-null float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


In [11]:
missing_df.head()

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
0,ORDB00336,2018-11-27,18:59:09,Dinner,TP,"[('Fish&Chips', 9), ('Shrimp', 2), ('Pasta', 5)]",560.5,-37.816186,145.012942,0,12.129,16.144210
1,ORDJ03182,2018-03-30,10:52:23,Breakfast,NaN,"[('Coffee', 6), ('Cereal', 1)]",66.0,-37.810246,144.961350,0,8.501,11.384186
2,ORDC01032,2018-09-18,16:37:10,Dinner,NS,"[('Salmon', 6), ('Shrimp', 4), ('Pasta', 6), (...",837.0,-37.806882,144.957889,0,8.110,13.893630
3,ORDX03103,2018-11-21,18:38:52,Dinner,BK,"[('Shrimp', 8), ('Salmon', 9), ('Fish&Chips', ...",1188.5,-37.813348,144.935750,0,10.285,17.171047
4,ORDJ00175,2018-08-08,15:16:03,Lunch,NaN,"[('Burger', 7), ('Steak', 4)]",397.0,-37.806441,144.941329,0,NaN,13.082562


In [12]:
missing_df.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,450.000000,450.000000
mean,500.571100,-37.812380,144.966818,0.064000,8.640987,13.713170
std,265.200107,0.007580,0.020441,0.244998,1.609263,2.710950
min,59.500000,-37.829848,144.926723,0.000000,3.640000,4.224023
25%,300.000000,-37.818463,144.953097,0.000000,7.744250,12.641206
50%,429.000000,-37.812833,144.963767,0.000000,8.677000,13.986931
75%,683.250000,-37.805928,144.978163,0.000000,9.665500,15.344647
max,1444.000000,-37.794547,145.018407,1.000000,13.559000,19.901359


In [ ]:
sum(missing_df['branch_code'].isnull())

In [ ]:
missing_df[missing_df['branch_code'] == 'NS']['order_id'].unique()

In [ ]:
missing_df[missing_df['order_id'].str.startswith('ORDB')]

In [ ]:
def check_branch_code(order_id, branch_code):
    if pd.isnull(branch_code):
        if order_id.startswith('ORDA') or order_id.startswith('ORDK') or order_id.startswith('ORDX'):
            return 'BK'
        elif order_id.startswith('ORDB') or order_id.startswith('ORDJ') or order_id.startswith('ORDY'):
            return 'TP'
        elif order_id.startswith('ORDC') or order_id.startswith('ORDI') or order_id.startswith('ORDZ'):
            return 'NS'
    return branch_code

missing_df['branch_code'] = missing_df.apply(lambda row:check_branch_code(row['order_id'], row['branch_code']),axis=1)

In [ ]:
# node.head()

In [ ]:
# print(node['lat'].max())
# print(node['lat'].min())
# print(node['lon'].max())
# print(node['lon'].min())

In [ ]:
outlier_df.shape

In [10]:
outlier_df.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.00000,500.000000,500.000000,500.00000,500.000000,500.000000
mean,487.88090,-37.812460,144.968418,0.04200,8.627326,13.703572
std,264.30641,0.007714,0.020821,0.20079,1.686663,2.692274
min,51.00000,-37.836908,144.894564,0.00000,3.758000,5.354800
25%,283.68750,-37.818475,144.953418,0.00000,7.684000,12.417850
50%,448.75000,-37.812848,144.965990,0.00000,8.586500,13.947467
75%,636.52500,-37.805937,144.982151,0.00000,9.619250,15.140462
max,1486.00000,-37.791484,145.018679,1.00000,19.900000,23.909199


In [ ]:
bp = outlier_df.boxplot(figsize=(12,10))
bp

In [ ]:
# 144.9631608,-37.8142177,144.9631608,-37.8142177

outlier_df.boxplot('order_price', by='order_type', figsize=(10,6))

In [ ]:
outlier_df.boxplot('delivery_fee', by='branch_code', figsize=(12,8))

In [ ]:
outlier_df['date'] = pd.to_datetime(outlier_df['date'])
outlier_df['time'] = pd.to_datetime(outlier_df['time'])

In [ ]:
def check_for_weekend(order_date):
    day = order_date.weekday()
    if day < 5:
        return 0
    else:
        return 1

outlier_df['weekend'] = outlier_df.apply(lambda row:check_for_weekend(row['date']),axis=1)

In [ ]:
outlier_tp_branch = outlier_df[outlier_df['branch_code'] == 'TP']
outlier_bk_branch = outlier_df[outlier_df['branch_code'] == 'BK']
outlier_ns_branch = outlier_df[outlier_df['branch_code'] == 'NS']

In [ ]:
import matplotlib.pyplot as plt

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)
n, bins, patches = ax1.hist(outlier_tp_branch['delivery_fee'], color='r')
ax1.set_xlabel('Branch TP Delivery Fee')
ax1.set_ylabel('Frequency')


fig2 = plt.figure()
ax2 = fig2.add_subplot(1, 1, 1)
n, bins, patches = ax2.hist(outlier_bk_branch['delivery_fee'])
ax2.set_xlabel('Branch BK Delivery Fee')
ax2.set_ylabel('Frequency')

fig3 = plt.figure()
ax3 = fig3.add_subplot(1, 1, 1)
n, bins, patches = ax3.hist(outlier_ns_branch['delivery_fee'], color='black')
ax3.set_xlabel('Branch NS Delivery Fee')
ax3.set_ylabel('Frequency')




In [ ]:
# outlier_tp_branch.boxplot('delivery_fee', by='order_type', figsize=(12,8))
# outlier_tp_branch.boxplot('delivery_fee', by='weekend', figsize=(12,8))
# outlier_bk_branch.boxplot('delivery_fee', by='order_type', figsize=(12,8))
# outlier_bk_branch.boxplot('delivery_fee', by='weekend', figsize=(12,8))
# outlier_ns_branch.boxplot('delivery_fee', by='order_type', figsize=(12,8))
# outlier_ns_branch.boxplot('delivery_fee', by='weekend', figsize=(12,8))

In [ ]:
outlier_df.loc[outlier_df['branch_code'] == 'TP','Predicted_Delivery_Fee'] =l0.predict(outliers[outlier_df['branch_code'] == 'TP'][['distance_to_customer_KM','weekend','order_type']])

sns.residplot(outlier_df[outlier_df['branch_code']=='TP']['Predicted_Delivery_Fee'],outlier_df[outlier_df['branch_code']=='TP']['delivery_fee'],lowess=True,line_kws={'color': 'red'} )
plt.xlabel('Fitted Values')
plt.ylabel('Fare ')
plt.title('Residual VS Fitted for Fare of Uber Type 0')

In [ ]:
outlier_df.head()

In [ ]:
outlier_df.boxplot('distance_to_customer_KM', figsize=(12,8))

In [ ]:
outlier_df.boxplot('distance_to_customer_KM', by='branch_code', figsize=(12,8))